In [121]:
import numpy as np
import sympy as sp

In [122]:
pi = 4.0 * np.arctan(1.0)
planck = 6.62606957e-34         # J s
avogadro = 6.0221413e23
kb = 1.3806488e-23              # J K^-1
speed_of_light = 299792458.0    # m s^-1
amu = 1.660538921e-27           # kg
gas_constant = avogadro*kb
hbar = planck/(2.0*pi)
global_eps = 1.0e-10

# Calculation of thermodynamic data

This is done in the end of all qce iterations. </br>
The partition functions, volumes and populations are calculated for each molecule at each temperature. </br>
They are then used for the calculation of the thermodynamic data.

## calculation of the system partition function
Attention: This subroutine is missing the summand arising from the particle </br>
indistinguishability.

$$ Q_{\text{sys}}(T) = \prod_{i=1}^{N_{\text{clust}}} Q(T)^{N_{\text{clust}}(T)} $$
$$ \ln{Q_{\text{sys}}(T)} = \sum_{i=1}^{N_{\text{clust}}} N_{\text{clust}}(T) \ln{Q(T)} $$

In [123]:
def calc_lnq_sys(ntemp, nclust, pop, ln_clust, lnq_sys):
    # ntemp     : input     - number of temperatures at which the system is simulated
    # nclust    : input     - number of clusters in the system
    # pop       : input     - population of each cluster at each temperature
    # ln_clust  : input     - ln of the population of each cluster
    # lnq_sys   : output    - ln of the partition function of the system
    
    lnq_sys = np.zeros(ntemp)
    for i in range(ntemp):
        for j in range(nclust):
            lnq_sys[i] += pop[i][j] * ln_clust[i][j]
            
    return lnq_sys

#### Tests

In [124]:
ntemp = 5
nclust = 4

pop = np.array([[0.5, 0.3, 0.1, 0.1],
                [0.4, 0.3, 0.2, 0.1],
                [0.3, 0.3, 0.3, 0.1],
                [0.2, 0.3, 0.4, 0.1],
                [0.1, 0.3, 0.5, 0.1]])

ln_clust = np.array([[-5.2, 3.8, 2.3, 1.3],
                     [-4.2, 3.3, 2.8, 1.3],
                     [-3.2, 3.3, 3.3, 1.3],
                     [-2.2, 3.3, 4.3, 1.3],
                     [-1.2, 3.3, 5.3, 1.3]])

lnq_sys = np.zeros(ntemp)

lnq_sys = calc_lnq_sys(ntemp, nclust, pop, ln_clust, lnq_sys)
print(lnq_sys)

[-1.10000000e+00 -3.33066907e-16  1.15000000e+00  2.40000000e+00
  3.65000000e+00]


## Add the particle indistinguishability 

$$ Q_{\text{sys}}(T) = \prod_{i=1}^{N_{\text{clust}}} \frac{1}{N!}Q(T)^{N_{\text{clust}}(T)} $$
$$ \ln{Q_{\text{sys}}(T)} = \sum_{i=1}^{N_{\text{clust}}} N_{\text{clust}}(T) \ln{Q(T)} - \sum_{i=1}^{N_{\text{clust}}} \ln{N!}$$

In [125]:
def add_lnq_indi(ntemp, nclust, pop, lnq_sys):
    # ntemp     : input     - number of temperatures at which the system is simulated
    # nclust    : input     - number of clusters in the system
    # pop       : input     - population of each cluster at each temperature
    # lnq_sys   : input     - ln of the partition function of the system
    
    lnq_indi = np.zeros(ntemp)
    for i in range(ntemp):
        for j in range(nclust):
            lnq_sys[i] += - sp.log(sp.factorial(pop[i][j]))
            
    return lnq_indi

#### Tests

In [126]:
ntemp = 5
nclust = 4

pop = np.array([[0.5, 0.3, 0.1, 0.1],
                [0.4, 0.3, 0.2, 0.1],
                [0.3, 0.3, 0.3, 0.1],
                [0.2, 0.3, 0.4, 0.1],
                [0.1, 0.3, 0.5, 0.1]])

lnq_sys = [-1.1, -3.33066907e-16,  1.15, 2.4,  3.65]

add_lnq_indi(ntemp, nclust, pop, lnq_sys)
print(lnq_sys)

[-0.771298070337215, 0.363034254943387, 1.52439686978342, 2.76303425494339, 3.97870192966279]


## Calculation of the Helmholtz energy
$$ \begin{aligned} A(T) &= -kT \ln{Q} \\
                        &= U - TS \end{aligned}$$

In [127]:
def calculate_helmholtz_energy(ntemp, temp, lnq):
    # ntemp     : input     - number of temperatures at which the system is simulated
    # temp      : input     - temperature of the system
    # lnq       : input     - ln of the partition function of the system
    # A         : output    - Helmholtz energy of the system
    
    A = np.zeros(ntemp)
    for i in range(ntemp):
        A[i] = - kb*temp[i] * lnq[i]
        
    return A

#### Tests

In [128]:
ntemp = 8
temp = np.array([100.15, 200.0, 300.0, 406.05, 500.0, 604.0, 730.4, 850.0])
lnq = np.array([100.03, 2.45, 3.2, 4.99, 5.04, 6.0, 7.089, 8.54])

A = calculate_helmholtz_energy(ntemp, temp, lnq)
print(A)

[-1.38313459e-19 -6.76517912e-21 -1.32542285e-20 -2.79745610e-20
 -3.47923498e-20 -5.00347125e-20 -7.14873109e-20 -1.00221296e-19]


## Calculation of Gibbs free energy

$$\begin{aligned} G &= - \frac{\ln{Q}}{\beta} + PV \\
                    &= - \frac{\ln{\frac{1}{N!}q^N}}{\beta} + PV \\
                    &= k_{\text{B}}T \bigg( \ln{N!} - N \ln{q} \bigg) +PV \\
                    &= A(T) + PV \end{aligned}$$

In [129]:
def calculate_gibbs_enthalpy(ntemp, temp, lnq, press, vol):
    # ntemp     : input     - number of temperatures at which the system is simulated
    # temp      : input     - temperature of the system
    # lnq       : input     - ln of the partition function of the system
    # G         : output    - Gibbs free energy of the system
    
    a = calculate_helmholtz_energy(ntemp, temp, lnq)
    
    G = np.zeros(ntemp)
    for i in range(ntemp):
        G[i] = a[i] + press*vol[i]
        
    return G

#### Tests

In [130]:
ntemp = 12
temp = np.array([100.15, 233.4, 512.0, 406.05, 530.0, 604.0, 730.4, 850.0, 900.0, 423.0, 1100.0, 1200.8])
lnq = np.array([1.03e-2, -2.45, 3.2, 1.99, 5.04, 53.0, -7.5, 8.54, 9.0, 10.0, 11.0, -12.0])
press = 1.01325
vol = np.array([0.5, 0.6, 4.7, 0.8, 0.9, 1.0, 1.1, 61.2, 1.3, 1.4, 1.5, 12.6])

G = calculate_gibbs_enthalpy(ntemp, temp, lnq, press, vol)
print(G)

[ 0.506625  0.60795   4.762275  0.8106    0.911925  1.01325   1.114575
 62.0109    1.317225  1.41855   1.519875 12.76695 ]


## Calculation of internal energy

$$ \begin{aligned} U(T) &= - \frac{\partial}{\partial \beta} \ln{Q} \\
                        &= k_{\text{B}}T^2 \ln{Q} \end{aligned}$$

In [131]:
def calculate_internal_energy(ntemp, temp, dlnq):
    # ntemp     : input     - number of temperatures at which the system is simulated
    # temp      : input     - temperature of the system
    # lnq       : input     - ln of the partition function of the system
    # U         : output    - Internal energy of the system
    
    U = np.zeros(ntemp)
    for i in range(ntemp):
        U[i] = kb*temp[i]**2 * dlnq[i]
        
    return U

In [132]:
ntemp = 21
temp = np.array([100.15, 233.4, 512.0, 406.05, 530.0, 604.0, 730.4, 850.0, 900.0, 423.0, 1100.0, 1200.8, 100.15, 233.4, 512.0, 406.05, 530.0, 604.0, 730.4, 850.0, 900.0])
dlnq = np.array([1.03e-2, -2.45, 3.2, 1.99, 5.04, 53.0, -7.5, 8.54, 9.0, 10.0, 11.0, -12.0, 1.03e-2, -2.45, 3.2, 1.99, 5.04, 53.0, -7.5, 8.54, 9.0])

U = calculate_internal_energy(ntemp, temp, dlnq)
print(U)

[ 1.42633767e-21 -1.84268461e-18  1.15817216e-17  4.52997000e-18
  1.95463421e-17  2.66951869e-16 -5.52415699e-17  8.51881019e-17
  1.00649298e-16  2.47038109e-17  1.83764355e-16 -2.38894320e-16
  1.42633767e-21 -1.84268461e-18  1.15817216e-17  4.52997000e-18
  1.95463421e-17  2.66951869e-16 -5.52415699e-17  8.51881019e-17
  1.00649298e-16]


## Calculation of enthalpy

$$ H(T) = U(T) + PV $$

In [133]:
def calculate_enthalpy(ntemp, temp, dlnq, vol, press):
    # ntemp     : input     - number of temperatures at which the system is simulated
    # temp      : input     - temperature of the system
    # lnq       : input     - ln of the partition function of the system
    # vol       : input     - volume of the system
    # press     : input     - pressure of the system
    # heat_cap  : output    - Enthalpy of the system
    
    u = calculate_internal_energy(ntemp, temp, dlnq)
    
    h = np.zeros(ntemp)
    for i in range(ntemp):
        h[i] = u[i] + press*vol[i]
        
    return h

#### Tests

In [137]:
ntemp = 18
temp = np.array([412.25, 123.4, 512.0, 406.05, 530.0, 604.0, 730.4, 850.0, 900.0, 423.0, 1100.0, 1200.8, 100.15, 233.4, 512.0, 406.05, 530.0, 604.0])
dlnq = np.array([-2.4, 38.5, 20.43, 1.99, 5.04, 53.0, -7.5, 8.54, 9.0, 10.0, 11.0, -12.0, 1.03e-2, -2.45, 3.2, 1.99, 5.04, 53.0])
vol = np.array([0.33, 23.5, 17.32, 0.43, 8.3, 1.0, 1.1, 61.2, 1.3, 1.4, 1.5, 12.6, 0.33, 23.5, 17.32, 0.43, 8.3, 1.0])
press = 1e-20

h = calculate_enthalpy(ntemp, temp, dlnq, vol, press)
print(h)

[-5.62809240e-18  8.32920629e-18  7.41152536e-17  4.53427000e-18
  1.96293421e-17  2.66961869e-16 -5.52305699e-17  8.58001019e-17
  1.00662298e-16  2.47178109e-17  1.83779355e-16 -2.38768320e-16
  4.72633767e-21 -1.60768461e-18  1.17549216e-17  4.53427000e-18
  1.96293421e-17  2.66961869e-16]


## Calculation of entropy

$$ \begin{aligned} S &= k_{\text{B}} \bigg( \ln{Q} + \beta \frac{\partial}{\partial \beta} \ln{Q} \bigg) \\
                     &= \frac{U(T) - A(T)}{T} \end{aligned}$$

In [138]:
def calculate_entropy(ntemp, temp, lnq, dlnq):
    # ntemp     : input     - number of temperatures at which the system is simulated
    # temp      : input     - temperature of the system
    # lnq       : input     - ln of the partition function of the system
    # dlnq      : input     - derivative of ln of the partition function of the system
    # S         : output    - Entropy of the system
    
    u = calculate_internal_energy(ntemp, temp, dlnq)
    a = calculate_helmholtz_energy(ntemp, temp, lnq)
    
    S = np.zeros(ntemp)
    for i in range(ntemp):
        S[i] = (u[i] - a[i])/temp[i]
        
    return S

#### Tests

In [148]:
ntemp = 33
temp = np.array([412.25, 123.4, 512.0, 406.05, 530.0, 604.0, 730.4, 850.0, 900.0, 423.0, 1100.0, 1200.8, 100.15, 233.4, 512.0, 406.05, 530.0, 604.0, 730.4, 850.0, 900.0, 423.0, 1100.0, 1200.8, 100.15, 233.4, 512.0, 406.05, 530.0, 604.0, 730.4, 850.0, 900.0])
lnq = np.array([-2.4, 38.5, 20.43, 1.99, 5.04, 53.0, -7.5, 8.54, 9.0, 10.0, 11.0, -12.0, 1.03e-2, -2.45, 3.2, 1.99, 5.04, 53.0, -7.5, 8.54, 9.0, 10.0, 11.0, -12.0, 1.03e-2, -2.45, 3.2, 1.99, 5.04, 53.0, -7.5, 8.54, 9.0])
dlnq = np.array([1.03e-2, -2.45, 3.2, 1.99, 5.04, 53.0, -7.5, 8.54, 9.0, 10.0, 11.0, -12.0, 1.03e-2, -2.45, 3.2, 1.99, 5.04, 53.0, -7.5, 8.54, 9.0, 10.0, 11.0, -12.0, 1.03e-2, -2.45, 3.2, 1.99, 5.04, 53.0, -7.5, 8.54, 9.0])

S = calculate_entropy(ntemp, temp, lnq, dlnq)
print(S)
print(len(S))

[ 2.54891930e-23 -3.64256573e-21  2.29026165e-20  1.11836626e-20
  3.69494754e-20  4.42705038e-19 -7.57354899e-20  1.00339204e-19
  1.11956811e-19  5.85395091e-20  1.67210376e-19 -1.99111647e-19
  1.43842205e-23 -7.92878993e-21  2.26647307e-20  1.11836626e-20
  3.69494754e-20  4.42705038e-19 -7.57354899e-20  1.00339204e-19
  1.11956811e-19  5.85395091e-20  1.67210376e-19 -1.99111647e-19
  1.43842205e-23 -7.92878993e-21  2.26647307e-20  1.11836626e-20
  3.69494754e-20  4.42705038e-19 -7.57354899e-20  1.00339204e-19
  1.11956811e-19]
33
